In [21]:
import os

os.system('export CUDNN_PATH="/home/tryptag/kiran/unetv20/lib/python3.10/site-packages/nvidia/cudnn"')
os.system('export LD_LIBRARY_PATH=${CUDNN_PATH}/lib')

0

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [23]:
def double_conv_block(x, n_filters):
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal")(x)
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding="same", activation="relu", kernel_initializer="he_normal")(x)

    return x

# Downsample Block
def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)

    return f, p


def upsample_block(x, conv_features, n_filters):
    # upsample
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    # concatenate
    x = layers.concatenate([x, conv_features])
    # dropout
    x = layers.Dropout(0.3)(x)
    # Conv2D twice with ReLU activation
    x = double_conv_block(x, n_filters)

    return x

def build_unet_model():
    inputs = layers.Input(shape=(320, 320, 1))

    f1, p1 = downsample_block(inputs, 64)
    f2, p2 = downsample_block(p1, 128)
    f3, p3 = downsample_block(p2, 256)
    f4, p4 = downsample_block(p3, 512)

    bottleneck = double_conv_block(p4, 1024)

    u6 = upsample_block(bottleneck, f4, 512)

    u7 = upsample_block(u6, f3, 256)

    u8 = upsample_block(u7, f2, 128)

    u9 = upsample_block(u8, f1, 64)

    outputs = layers.Conv2D(1, 1, padding="same", activation = "sigmoid")(u9)

    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")
    
    return unet_model

In [24]:
images = np.load('test_images_TP.npy')
ground_truth_masks = np.load('test_masks_TP.npy')

## Models

### 23 May 2024
Uses the random test-train split and hence the model was trained on random images. This experiment is therefore DISCARDED.


### 30 May 2024
Same phase channel images for Tryptag and Ilastik sets.

BUT MISSING masks in Tryptag

### 14 June 2024
Phase Channels images are not the same for Tryp and Ilastik

### 20 June 2024
Same phase channel images for Tryptag and Ilastik sets.

Uses Tryptag masks that are good

### 27 June 2024
Same experiment as 20 June 2024 (mistake)

### 28 June 2024 (GDL)
Uses the Generalized Dice Loss implementation that Ellen created as the loss function

### 28 June 2024 (log(cosh(GDL)))
Use the $\log (\cosh (GDL) )$ as the loss function.

Modification of what is used by Jadon et al.

In [68]:
models = {
    "23May2024" : {
            "Ilastik Masks" : '',
            "Tryptag Masks" : '',
            "Composite Masks" : ''
    },
    "30May2024" : {
            "Ilastik Masks" : '/datapot/tryptag/kiran/CellSegmentation_Backup14June2024/unet_30May24_Ilastik/N_EPOCHS_1000/LearningRate_5e-05/BatchSize_24/unet_model_30May24_Ilastik.weights.h5',
            "Tryptag Masks" : '/datapot/tryptag/kiran/CellSegmentation_Backup14June2024/unet_30May24_Tryptag/N_EPOCHS_1000/LearningRate_5e-05/BatchSize_24/unet_model_30May24_Tryptag.weights.h5',
            "Composite Masks" : '/datapot/tryptag/kiran/CellSegmentation_Backup14June2024/unet_30May24_Combined/N_EPOCHS_1000/LearningRate_0.0001/BatchSize_24/unet_model_30May24_Combined.weights.h5' 
    },
    "14June2024" : {
            "Ilastik Masks" : '/datapot/tryptag/kiran/CellSegmentation_Backup14June2024/unet_30May24_Ilastik/N_EPOCHS_1000/LearningRate_5e-05/BatchSize_24/unet_model_30May24_Ilastik.weights.h5',
            "Tryptag Masks" : '/datapot/tryptag/kiran/unet_14Juni2024_Tryptag/N_EPOCHS_2000/LearningRate_0.0001/BatchSize_24/unet_model_14Juni2024_Tryptag.weights.h5',
            "Composite Masks" : '/datapot/tryptag/kiran/unet_14Juni2024_Combined/N_EPOCHS_2000/LearningRate_1e-05/BatchSize_24/unet_model_14Juni2024_Combined.weights.h5'
    },
    "20June2024" : {
            "Ilastik Masks" : '/datapot/tryptag/kiran/Results/unet_20Juni2024_Ilastik/N_EPOCHS_2000/LearningRate_0.0001/BatchSize_24/unet_model_20Juni2024_Ilastik.weights.h5',
            "Tryptag Masks" : '/datapot/tryptag/kiran/Results/unet_20Juni2024_Tryptag/N_EPOCHS_1000/LearningRate_0.0001/BatchSize_24/unet_model_20Juni2024_Tryptag.weights.h5',
            "Composite Masks" : '/datapot/tryptag/kiran/Results/unet_20Juni2024_Combined/N_EPOCHS_1000/LearningRate_5e-05/BatchSize_24/unet_model_20Juni2024_Combined.weights.h5'
    },
    "27June2024" : {
            "Ilastik Masks" : '/datapot/tryptag/kiran/Results/unet_27Juni2024_Ilastik/N_EPOCHS_1000/LearningRate_5e-05/BatchSize_24/unet_model_27Juni2024_Ilastik.weights.h5',
            "Tryptag Masks" : '/datapot/tryptag/kiran/Results/unet_27Juni2024_Tryptag/N_EPOCHS_2000/LearningRate_0.0001/BatchSize_24/unet_model_27Juni2024_Tryptag.weights.h5',
            "Composite Masks" : '/datapot/tryptag/kiran/Results/unet_27Juni2024_Combined/N_EPOCHS_1000/LearningRate_5e-05/BatchSize_24/unet_model_27Juni2024_Combined.weights.h5'
    },
    "28June2024_GDL": {
            "Ilastik Masks" : '/datapot/tryptag/kiran/Results/unet_28Juni2024_GDL_Ilastik/N_EPOCHS_2000/LearningRate_0.0001/BatchSize_24/unet_model_28Juni2024_GDL_Ilastik.weights.h5',
            "Tryptag Masks" : '/datapot/tryptag/kiran/Results/unet_28Juni2024_GDL_Tryptag/N_EPOCHS_2000/LearningRate_0.0001/BatchSize_24/unet_model_28Juni2024_GDL_Tryptag.weights.h5',
            "Composite Masks" : '/datapot/tryptag/kiran/Results/unet_28Juni2024_GDL_Combined/N_EPOCHS_1000/LearningRate_0.0001/BatchSize_24/unet_model_28Juni2024_GDL_Combined.weights.h5'
    },
    "28June2024_LogCoshGDL": {
            "Ilastik Masks" : '/datapot/tryptag/kiran/Results/unet_28Juni2024_LogCoshGDL_Ilastik/N_EPOCHS_2000/LearningRate_5e-05/BatchSize_24/unet_model_28Juni2024_LogCoshGDL_Ilastik.weights.h5',
            "Tryptag Masks" : '/datapot/tryptag/kiran/Results/unet_28Juni2024_LogCoshGDL_Tryptag/N_EPOCHS_1000/LearningRate_5e-05/BatchSize_24/unet_model_28Juni2024_LogCoshGDL_Tryptag.weights.h5',
            "Composite Masks" : '/datapot/tryptag/kiran/Results/unet_28Juni2024_LogCoshGDL_Combined/N_EPOCHS_2000/LearningRate_0.0001/BatchSize_24/unet_model_28Juni2024_LogCoshGDL_Combined.weights.h5'
    },
}

In [78]:
models.keys()

dict_keys(['23May2024', '30May2024', '14June2024', '20June2024', '27June2024', '28June2024_GDL', '28June2024_LogCoshGDL'])

In [79]:
# best_models = {
#     "Ilastik Masks" : '/home/tryptag/kiran/CellSegmentation/unet_20Juni2024_Ilastik/N_EPOCHS_2000/LearningRate_0.0001/BatchSize_24/unet_model_20Juni2024_Ilastik.weights.h5',
#     "Tryptag Masks" : '/home/tryptag/kiran/CellSegmentation/unet_20Juni2024_Tryptag/N_EPOCHS_1000/LearningRate_0.0001/BatchSize_24/unet_model_20Juni2024_Tryptag.weights.h5',
#     "Composite Masks" : '/home/tryptag/kiran/CellSegmentation/unet_20Juni2024_Combined/N_EPOCHS_1000/LearningRate_5e-05/BatchSize_24/unet_model_20Juni2024_Combined.weights.h5',
# }

experiment_date = '28June2024_LogCoshGDL'
best_models = models[experiment_date]
best_models

{'Ilastik Masks': '/datapot/tryptag/kiran/Results/unet_28Juni2024_LogCoshGDL_Ilastik/N_EPOCHS_2000/LearningRate_5e-05/BatchSize_24/unet_model_28Juni2024_LogCoshGDL_Ilastik.weights.h5',
 'Tryptag Masks': '/datapot/tryptag/kiran/Results/unet_28Juni2024_LogCoshGDL_Tryptag/N_EPOCHS_1000/LearningRate_5e-05/BatchSize_24/unet_model_28Juni2024_LogCoshGDL_Tryptag.weights.h5',
 'Composite Masks': '/datapot/tryptag/kiran/Results/unet_28Juni2024_LogCoshGDL_Combined/N_EPOCHS_2000/LearningRate_0.0001/BatchSize_24/unet_model_28Juni2024_LogCoshGDL_Combined.weights.h5'}

In [80]:
# for keys in best_models:
#     model_weights_path = best_models[keys]
#     unet_model = build_unet_model()
#     metrics = [tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)]
#     unet_model.compile(optimizer=Adam(), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
#                                    metrics=metrics)
#     unet_model.load_weights(model_weights_path)
#     test_loss, test_iou = unet_model.evaluate(tf.stack(images), tf.stack(ground_truth_masks))
#     print("\n", keys)
#     print("Test loss : ", test_loss)
#     print("Test IoU : ", test_iou)

In [81]:
ilastik_unet = build_unet_model()
metrics = [tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)]
ilastik_unet.compile(optimizer=Adam(), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                   metrics=metrics)
ilastik_unet.load_weights(best_models["Ilastik Masks"])
ilastik_predictions = ilastik_unet.predict(tf.stack(images))
ilastik_predictions = ilastik_predictions > 0.5

7/7 ━━━━━━━━━━━━━━━━━━━━ 54s 8s/step


In [82]:
tryptag_unet = build_unet_model()
metrics = [tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)]
tryptag_unet.compile(optimizer=Adam(), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                   metrics=metrics)
tryptag_unet.load_weights(best_models["Tryptag Masks"])
tryptag_predictions = tryptag_unet.predict(tf.stack(images))
tryptag_predictions = tryptag_predictions > 0.5

7/7 ━━━━━━━━━━━━━━━━━━━━ 54s 8s/step


In [83]:
composite_unet = build_unet_model()
metrics = [tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)]
composite_unet.compile(optimizer=Adam(), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                   metrics=metrics)
composite_unet.load_weights(best_models["Composite Masks"])
composite_predictions = composite_unet.predict(tf.stack(images))
composite_predictions = composite_predictions > 0.5

7/7 ━━━━━━━━━━━━━━━━━━━━ 55s 8s/step


In [84]:
# for i in range(len(images)):
#     fig, axes = plt.subplots(1, 5, figsize=(20,5))
#     # fig, axes = plt.subplots(1, 3, figsize=(20,5))
#     axes[0].imshow(images[i])
#     axes[0].axis('off')
#     axes[0].set_title('Phase Contrast')
    
#     axes[1].imshow(ground_truth_masks[i])
#     axes[1].axis('off')
#     axes[1].set_title('Ground Truth')
    
#     axes[2].imshow(ilastik_predictions[i])
#     axes[2].axis('off')
#     axes[2].set_title('Ilastik Trained')
    
#     axes[3].imshow(tryptag_predictions[i])
#     axes[3].axis('off')
#     axes[3].set_title('Tryptag Trained')
    
#     axes[4].imshow(composite_predictions[i])
#     axes[4].axis('off')
#     axes[4].set_title('Trained on mixture')

#     plt.tight_layout()
#     plt.show()
#     fig.savefig(f'Comparison_20June2024/Images_and_Masks_{i}.png')
#     plt.close(fig)

In [85]:
np.save(f'/datapot/tryptag/kiran/Predictions_using_models/ilastik_predictions_{experiment_date}.npy', tf.squeeze(ilastik_predictions).numpy())

In [86]:
np.save(f'/datapot/tryptag/kiran/Predictions_using_models/tryptag_predictions_{experiment_date}.npy', tf.squeeze(tryptag_predictions).numpy())
np.save(f'/datapot/tryptag/kiran/Predictions_using_models/combined_predictions_{experiment_date}.npy', tf.squeeze(composite_predictions).numpy())